In [10]:
import sqlite3
import pandas as pd

## Read CSVS

In [11]:
# Read in csv data
slot_performance = pd.read_csv("../data/slot_performance.csv")
slot_performance.start_date = pd.to_datetime(slot_performance.start_date, format='%m/%d/%Y')
slot_performance.end_date = pd.to_datetime(slot_performance.end_date, format='%m/%d/%Y')

location = pd.read_csv("../data/location.csv")

In [12]:
slot_performance.head(3)

,employer_id,city_id,contract_id,start_date,end_date,renewed_flag,job_slots,total_contract_value,applications,apply_start_clicks,click_marketplace_value,job_listings
0,4277920,9767,2423796,2017-10-30,2017-11-30,1,15,995,215,255,15484.40,25
1,2372987,8585,2074128,2017-06-29,2017-07-29,0,50,1495,55,70,1719.30,50
2,1142693,6665,1924740,2017-05-02,2017-06-02,0,15,495,45,95,1195.75,20


In [13]:
location.head(3)

,city_id,city_name,state_id,state_name
0,1,Rockford,OH,Ohio
1,2,Sedgewickville,MO,Missouri
2,3,Whitehouse,OH,Ohio


## SQLite DB

In [14]:
con = sqlite3.connect('../data/glassdoor.db')

In [15]:
slot_performance.to_sql('slot_performance', con, if_exists='replace', index=False)
location.to_sql('location', con, if_exists='replace', index=False)

In [16]:
# Test location write
locs = pd.read_sql('SELECT * FROM location LIMIT 3', con)
locs.head()

,city_id,city_name,state_id,state_name
0,1,Rockford,OH,Ohio
1,2,Sedgewickville,MO,Missouri
2,3,Whitehouse,OH,Ohio


In [17]:
# Test slot performance write
sp = pd.read_sql('SELECT * FROM slot_performance LIMIT 3', con)
sp.head()

,employer_id,city_id,contract_id,start_date,end_date,renewed_flag,job_slots,total_contract_value,applications,apply_start_clicks,click_marketplace_value,job_listings
0,4277920,9767,2423796,2017-10-30 00:00:00,2017-11-30 00:00:00,1,15,995,215,255,15484.40,25
1,2372987,8585,2074128,2017-06-29 00:00:00,2017-07-29 00:00:00,0,50,1495,55,70,1719.30,50
2,1142693,6665,1924740,2017-05-02 00:00:00,2017-06-02 00:00:00,0,15,495,45,95,1195.75,20


In [18]:
del locs
del sp

## PART A: Write SQL queries to answer the following questions

Total Contract Value is defined as the total amount that customers committed to spend. Write a SQL query that returns the Total Contract Value ('total_contract_value' field) for each state (name) by month (using 'start_date') in the provided datasets.

In [19]:
q = '''
SELECT
    l.city_name,
    strftime('%Y-%m', p.start_date) AS year_month,
    SUM(p.total_contract_value) AS total_contract_value
FROM
    slot_performance AS p
JOIN
    location AS l
ON
    l.city_id = p.city_id
GROUP BY
    city_name,
    year_month
'''
cv = pd.read_sql(q, con)
cv

,city_name,year_month,total_contract_value
0,Abbott,2016-08,495
1,Abbott,2017-03,1245
2,Abbott,2017-07,1495
3,Aberdeen,2016-07,495
4,Aberdeen,2016-10,1495
...,...,...,...
33641,Zuehl,2017-12,995
33642,Zwingle,2016-06,1245
33643,Zwingle,2017-03,1245
33644,Zwingle,2017-06,1745


For all employers who purchased >1 product with Glassdoor, write a SQL query to return the 'job_slots' and 'click_marketplace_value' values for the second transaction by employer.

In [24]:
q = '''
WITH FOO as (
    SELECT
        employer_id,
        start_date
    FROM
        slot_performance
)
SELECT * FROM FOO
'''
q = '''
SELECT
    employer_id,
    start_date,
    ROW_NUMBER() OVER (
        PARTITION BY employer_id,
        ORDER BY start_date
    )
'''
ff = pd.read_sql(q, con)
ff

DatabaseError: Execution failed on sql '
SELECT
    employer_id,
    start_date,
    ROW_NUMBER() OVER (
        PARTITION BY employer_id,
        ORDER BY start_date
    )
': near "(": syntax error

In [23]:
sqlite3.sqlite_version

'3.22.0'

In [23]:
sqlite3.sqlite_version

'3.22.0'

## Close Connection

In [ ]:
con.close()